<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# TL;DR

1. In this lab scenario you will have a chance to compare performance of the classic RNN and LSTM on a toy example.
2. This toy example will show that maintaining memory over even 20 steps is non-trivial.
3. Finally, you will see how curriculum learning may allow to train a model on larger sequences.

# Problem definition

Here we consider a toy example, where the goal is to discriminate between two types of binary sequences:
* [Type 0] a sequence with exactly one zero (remaining entries are equal to one).
* [Type 1] a sequence full of ones,

We are especially interested in the performance of the trained models on discriminating between a sequence full of ones versus a sequence with leading zero followed by ones. Note that in this case the goal of the model is to output the first element of the sequence, as the label (sequence type) is fully determined by the first element of the sequence.

#Implementation

## Importing torch

Install `torch` and `torchvision`

In [1]:
!pip3 install torch torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
from typing import List
from tqdm import tqdm

torch.manual_seed(1)

## Understand dimensionality

Check the input and output specification [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) and [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). The following snippet shows how we can process
a sequence by LSTM and output a vector of size `hidden_dim` after reading
each token of the sequence.

In [ ]:
hidden_dim = 5
lstm = nn.LSTM(1, hidden_dim)  # Input sequence contains elements - vectors of size 1

# create a random sequence
sequence = [torch.randn(1) for _ in range(10)]

# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

for i, elem in enumerate(sequence):
  # we are processing only a single element of the sequence, and there
  # is only one sample (sequence) in the batch, the third one
  # corresponds to the fact that our sequence contains elemenents,
  # which can be treated as vectors of size 1
  out, hidden = lstm(elem.view(1, 1, 1), hidden)
  print(f'i={i} out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[[ 0.0396,  0.0791,  0.0660,  0.1606, -0.0123]]])
i=1 out=tensor([[[ 0.0548,  0.1250,  0.1118,  0.2201, -0.0197]]])
i=2 out=tensor([[[ 0.0631,  0.1464,  0.1355,  0.2458, -0.0244]]])
i=3 out=tensor([[[ 0.0883,  0.1290,  0.1077,  0.2996, -0.0124]]])
i=4 out=tensor([[[ 0.0515,  0.1840,  0.1855,  0.2289, -0.0334]]])
i=5 out=tensor([[[ 0.0656,  0.1654,  0.1625,  0.2604, -0.0362]]])
i=6 out=tensor([[[ 0.0399,  0.2011,  0.2175,  0.2154, -0.0571]]])
i=7 out=tensor([[[ 0.0299,  0.2115,  0.2359,  0.2058, -0.0774]]])
i=8 out=tensor([[[ 0.0605,  0.1727,  0.1826,  0.2573, -0.0583]]])
i=9 out=tensor([[[ 0.0483,  0.1949,  0.2132,  0.2282, -0.0604]]])
Final hidden state=tensor([[[ 0.0483,  0.1949,  0.2132,  0.2282, -0.0604]]]) cell state=tensor([[[ 0.1221,  0.5815,  0.5050,  0.5271, -0.0939]]])


## To implement

Process the whole sequence all at once by calling `lstm` only once and check that the output is exactly the same as above (remember to initialize the hidden state the same way).

In [ ]:
# #########################################################
#                    To implement
# #########################################################
sequence_torch = torch.stack(sequence)

hidden_lstm = (torch.zeros(1, 1, 5),
               torch.zeros(1, 1, 5))

# input_tensor shape: (L, N, H_in), L=sequence length, N=batch size, H_in=input_size
out, hidden_lstm = lstm(sequence_torch.view(10,1,1), hidden_lstm)
print(torch.allclose(hidden_lstm[0], hidden[0])  and torch.allclose(hidden_lstm[1], hidden[1]))

True


## Training a model

Below we define a very simple model, which is a single layer of LSTM, where the output in each time step is processed by relu followed by a single fully connected layer, the output of which is a single number. We are going
to use the number generated after reading the last element of the sequence,
which will serve as the logit for our classification problem.

In [3]:
class Model(nn.Module):

    def __init__(self, hidden_dim: int):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

Below is a training loop, where we only train on the two hardest examples.

In [4]:
SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

# 2
def generate_examples(sequence_len: int):
    sequences = []
    for _ in range(100):
        sequences.append(([0.] + [random.choice([0., 1.]) for _ in range(sequence_len - 1)], 0))
        sequences.append(([1.] * sequence_len, 1))
    return sequences


def eval_on_hard_examples(model: nn.Module) -> List[float]:
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')
        return logits


def train_model(hidden_dim: int, lr: float, num_steps:int = 10000, examples = HARD_EXAMPLES) -> None:
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    pbar = tqdm(range(num_steps))
    for step in pbar:
        if step % 100 == 0:
            logits = eval_on_hard_examples(model)
            pbar.set_postfix(logits=logits)

        for sequence, label in examples:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))

            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

In [ ]:
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

  0%|          | 37/10000 [00:00<01:35, 104.49it/s, logits=[tensor([-0.0075]), tensor([-0.0076])]]

Logits for hard examples=[tensor([-0.0075]), tensor([-0.0076])]


  1%|▏         | 137/10000 [00:00<00:39, 252.28it/s, logits=[tensor([0.0218]), tensor([0.0218])]]

Logits for hard examples=[tensor([0.0218]), tensor([0.0218])]


  2%|▏         | 241/10000 [00:01<00:31, 311.48it/s, logits=[tensor([0.0026]), tensor([0.0026])]]

...

 99%|█████████▊| 9852/10000 [00:32<00:00, 319.75it/s, logits=[tensor([0.0025]), tensor([0.0025])]]

Logits for hard examples=[tensor([0.0025]), tensor([0.0025])]


100%|█████████▉| 9952/10000 [00:33<00:00, 321.53it/s, logits=[tensor([0.0025]), tensor([0.0025])]]

Logits for hard examples=[tensor([0.0025]), tensor([0.0025])]


100%|██████████| 10000/10000 [00:33<00:00, 299.29it/s, logits=[tensor([0.0025]), tensor([0.0025])]]


## To implement

1. Check for what values of `SEQUENCE_LEN` the model is able to discriminate betweeh the two hard examples (after training).
2. Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.
3. Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.
4. Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

Note that for steps 2-4 you may need to change the value of `num_steps`.

In [5]:
#1
for SEQUENCE_LEN in range(2, 12, 1):
    print(f"SEQUENCE_LEN={SEQUENCE_LEN}")
    HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0), (SEQUENCE_LEN*[1.], 1)]
    train_model(hidden_dim=20, lr=0.01, num_steps=10000)

SEQUENCE_LEN=2


  0%|          | 1/10000 [00:00<1:08:16,  2.44it/s, logits=[tensor([-0.0188]), tensor([-0.0101])]]

Logits for hard examples=[tensor([-0.0188]), tensor([-0.0101])]


  2%|▏         | 153/10000 [00:00<00:40, 243.59it/s, logits=[tensor([0.0099]), tensor([0.0172])]]

Logits for hard examples=[tensor([0.0099]), tensor([0.0172])]


...

 99%|█████████▊| 9853/10000 [00:40<00:00, 279.37it/s, logits=[tensor([-14.0266]), tensor([11.0444])]]

Logits for hard examples=[tensor([-14.0266]), tensor([11.0444])]


 99%|█████████▉| 9939/10000 [00:41<00:00, 273.12it/s, logits=[tensor([-14.0252]), tensor([11.0523])]]

Logits for hard examples=[tensor([-14.0252]), tensor([11.0523])]


100%|██████████| 10000/10000 [00:41<00:00, 240.90it/s, logits=[tensor([-14.0252]), tensor([11.0523])]]


In [ ]:
print(f"Value for [Type 0] : {torch.sigmoid(torch.tensor(0.0013))} , Value for [Type 1] : {torch.sigmoid(torch.tensor(0.0013))}")

Value for [Type 0] : 0.5003250241279602 , Value for [Type 1] : 0.5003250241279602


In [ ]:
# 2
SEQUENCE_LEN = 10
train_model(hidden_dim=20, lr=0.01, num_steps=10000, examples=generate_examples(SEQUENCE_LEN))

  0%|          | 0/10000 [00:00<?, ?it/s, logits=[tensor([-0.1412]), tensor([-0.1415])]]

Logits for hard examples=[tensor([-0.1412]), tensor([-0.1415])]


  1%|          | 100/10000 [00:30<46:14,  3.57it/s, logits=[tensor([-10.7415]), tensor([11.2493])]]

Logits for hard examples=[tensor([-10.7415]), tensor([11.2493])]


  2%|▏         | 200/10000 [01:01<1:00:13,  2.71it/s, logits=[tensor([-11.2921]), tensor([11.7204])]]

...

 98%|█████████▊| 9800/10000 [57:04<01:10,  2.84it/s, logits=[tensor([-15.0374]), tensor([15.2553])]]

Logits for hard examples=[tensor([-15.0374]), tensor([15.2553])]


 99%|█████████▉| 9900/10000 [57:42<00:33,  2.99it/s, logits=[tensor([-15.0471]), tensor([15.2626])]]

Logits for hard examples=[tensor([-15.0471]), tensor([15.2626])]


100%|██████████| 10000/10000 [58:26<00:00,  2.85it/s, logits=[tensor([-15.0471]), tensor([15.2626])]]


In [ ]:
print(f"Value for [Type 0] : {torch.sigmoid(torch.tensor(-14.5929))} , Value for [Type 1] : {torch.sigmoid(torch.tensor(14.8864))}")

Value for [Type 0] : 4.596039104853844e-07 , Value for [Type 1] : 0.9999996423721313


In [ ]:
# 3
class RNNModel(nn.Module):
    def __init__(self, hidden_dim: int):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.rnn(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

def train_rnn_model(hidden_dim: int, lr: float, num_steps: int = 10000) -> None:
    model = RNNModel(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    pbar = tqdm(range(num_steps))
    for step in pbar:
        if step % 100 == 0:
            logits = eval_on_hard_examples(model)
            pbar.set_postfix(logits=logits)

        sequences = generate_examples(SEQUENCE_LEN)
        for sequence, label in sequences:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))

            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

SEQUENCE_LEN = 10
train_rnn_model(hidden_dim=20, lr=0.01, num_steps=1000)

  0%|          | 0/1000 [00:00<?, ?it/s, logits=[tensor([-0.0216]), tensor([-0.0216])]]

Logits for hard examples=[tensor([-0.0216]), tensor([-0.0216])]


 10%|█         | 100/1000 [00:55<08:13,  1.82it/s, logits=[tensor([0.0101]), tensor([0.0101])]]  

Logits for hard examples=[tensor([0.0101]), tensor([0.0101])]


 20%|██        | 200/1000 [01:54<05:53,  2.26it/s, logits=[tensor([0.0101]), tensor([0.0101])]]

...

 80%|████████  | 800/1000 [06:26<01:22,  2.42it/s, logits=[tensor([0.0088]), tensor([0.0088])]]

Logits for hard examples=[tensor([0.0088]), tensor([0.0088])]


 90%|█████████ | 900/1000 [07:08<00:38,  2.61it/s, logits=[tensor([0.0088]), tensor([0.0088])]]

Logits for hard examples=[tensor([0.0088]), tensor([0.0088])]


100%|██████████| 1000/1000 [07:49<00:00,  2.13it/s, logits=[tensor([0.0088]), tensor([0.0088])]]


In [ ]:
print(f"Value for [Type 0] : {torch.sigmoid(torch.tensor(0.0088))} , Value for [Type 1] : {torch.sigmoid(torch.tensor(0.0088))}")

Value for [Type 0] : 0.5022000074386597 , Value for [Type 1] : 0.5022000074386597


In [ ]:
# 4
def curriculum_learning(hidden_dim: int, lr: float, max_seq_len: int, num_steps: int = 10000) -> None:
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for seq_len in range(1, max_seq_len + 1):
        print(f"Train for SEQUENCE_LEN={seq_len}")
        SEQUENCE_LEN = seq_len

        pbar = tqdm(range(num_steps))
        for step in pbar:
            if step % 100 == 0:
              logits = eval_on_hard_examples(model)
              pbar.set_postfix(logits=logits)

        sequences = generate_examples(SEQUENCE_LEN)
        for sequence, label in sequences:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1,1,1))

            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

curriculum_learning(hidden_dim=20, lr=0.01, max_seq_len=20, num_steps=10000)

Train for SEQUENCE_LEN=1


 13%|█▎        | 1301/10000 [00:00<00:00, 12599.99it/s, logits=[tensor([0.1224]), tensor([0.1223])]]

Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits f

 13%|█▎        | 1301/10000 [00:00<00:00, 12599.99it/s, logits=[tensor([0.1224]), tensor([0.1223])]]

Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]
Logits for hard examples=[tensor([0.1224]), tensor([0.1223])]


...


Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.06

 75%|███████▍  | 7467/10000 [00:00<00:00, 14339.83it/s, logits=[tensor([-9.0663]), tensor([20.1499])]]

Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.066

100%|██████████| 10000/10000 [00:00<00:00, 13301.87it/s, logits=[tensor([-9.0663]), tensor([20.1499])]]


Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]
Logits for hard examples=[tensor([-9.0663]), tensor([20.1499])]


In [ ]:
print(f"Value for [Type 0] : {torch.sigmoid(torch.tensor(-9.0663))} , Value for [Type 1] : {torch.sigmoid(torch.tensor(20.1499))}")

Value for [Type 0] : 0.00011547969188541174 , Value for [Type 1] : 1.0
